## Cleaning the ReliefWeb textual data
---
**Purpose**: This notebook will show the steps which will be followed to clean the collected data from the webscrapping. The data should be in a csv format and the column referring to the date of publication should have been changed to a datetime type OR a datetime type column should have been appended to the csv file (this was accomplished in the previous notebook). The bare minimum of columns required for each csv file should be: date (in datetime type), title (object type), and the text from the article (object type). Later on the source, where the webscrapping was done, will be added to the file.  

This notebook will prepare a csv file for eventual merging with others for eventual analysis and data extraction. 

In [1]:
# Libraries 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

%matplotlib inline

## Read in the csv file
Read in the csv file. Take special care to remember to add the argument `parse_dates` with the column name of the column which has the dates in datetime format. This will allow it to be imported as a datetime type instead of an object. This argument will have to be used whenever a datetime type column is part of the csv file. 

In [2]:
# Read in data from reliefweb AFTER 2006
# Notice: after the csv name there is an additional argument 
    
reliefweb06_df= pd.read_csv("../data/reliefweb_floods_after06.csv",
                            parse_dates = ["datetime"])

### Examine the dataframe
Take a quick look at the imported csv file. Check for null values, number of rows, and data types. 

In [3]:
# check
reliefweb06_df.head()

,title,date,body,url,datetime
0,Situation report on Flood/Heavy Rain fall as o...,21 Oct 2019,\nMinistry of Home Affairs\nDisaster Managemen...,https://reliefweb.int/node/3365543,2019-10-21
1,India: Monsoon Rains and Floods Emergency Plan...,18 Oct 2019,\nA. SITUATION ANALYSIS\nDescription of the di...,https://reliefweb.int/node/3354153,2019-10-18
2,India: Monsoon Floods Emergency Plan of Action...,14 Sep 2019,\nA. SITUATION ANALYSIS\nDescription of the di...,https://reliefweb.int/node/3303423,2019-09-14
3,Situation report on Flood/Heavy Rainfall as on...,05 Sep 2019,\n1. Forecast\ni. IMD\n\n05 September (Day 1):...,https://reliefweb.int/node/3292673,2019-09-05
4,Situation report on Flood/Heavy Rain fall as o...,02 Sep 2019,\n1. Forecast\ni. IMD\n• 02 September (Day 1):...,https://reliefweb.int/node/3287208,2019-09-02


In [4]:
# check types and non-null value count
reliefweb06_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 5 columns):
title       344 non-null object
date        344 non-null object
body        344 non-null object
url         344 non-null object
datetime    344 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 13.6+ KB


In [5]:
# double check for nulls
reliefweb06_df.isnull().sum()

title       0
date        0
body        0
url         0
datetime    0
dtype: int64

After the initial look at the dataframe, I can see there are no null values for any of the rows. There are 344 rows of data pulled from reliefweb.int which are the articles after 2006. All the columns, except the datetime are object (string) types which makes sense for this being textual data. The datetime is a datetime type which was created from the date (string) in the previous notebook.

In [6]:
# check how the datetime data is formatted
reliefweb06_df["datetime"][1]

Timestamp('2019-10-18 00:00:00')

### Source column 
A column named "source" will be created. This column will show from where the data comes.   Since this entire csv file is from one source (reliefweb.int) the column will be created and "reliefweb" will be inserted for each observation.

In [7]:
# Create column for the source of this data
reliefweb06_df["source"] = "reliefweb"

# check
reliefweb06_df.head()

,title,date,body,url,datetime,source
0,Situation report on Flood/Heavy Rain fall as o...,21 Oct 2019,\nMinistry of Home Affairs\nDisaster Managemen...,https://reliefweb.int/node/3365543,2019-10-21,reliefweb
1,India: Monsoon Rains and Floods Emergency Plan...,18 Oct 2019,\nA. SITUATION ANALYSIS\nDescription of the di...,https://reliefweb.int/node/3354153,2019-10-18,reliefweb
2,India: Monsoon Floods Emergency Plan of Action...,14 Sep 2019,\nA. SITUATION ANALYSIS\nDescription of the di...,https://reliefweb.int/node/3303423,2019-09-14,reliefweb
3,Situation report on Flood/Heavy Rainfall as on...,05 Sep 2019,\n1. Forecast\ni. IMD\n\n05 September (Day 1):...,https://reliefweb.int/node/3292673,2019-09-05,reliefweb
4,Situation report on Flood/Heavy Rain fall as o...,02 Sep 2019,\n1. Forecast\ni. IMD\n• 02 September (Day 1):...,https://reliefweb.int/node/3287208,2019-09-02,reliefweb


## Processing the text data
Convert all text to lowercase and replace line breaks with spaces. This will be done for the title and body of each row. 

In [8]:
def text_processing(df, column): 
    # Change all text to lowercase for movies dataframe body 
    df[column]= df[column].map(lambda x: x.lower())

    # Change line breaks into a space
    df[column] = df[column].replace(r"[\r\n]+", " ", regex=True)

In [9]:
# processing the title and body of each row of the dataframe
text_processing(reliefweb06_df, "body")
text_processing(reliefweb06_df, "title")

In [10]:
# check
reliefweb06_df.head()

,title,date,body,url,datetime,source
0,situation report on flood/heavy rain fall as o...,21 Oct 2019,ministry of home affairs disaster management ...,https://reliefweb.int/node/3365543,2019-10-21,reliefweb
1,india: monsoon rains and floods emergency plan...,18 Oct 2019,a. situation analysis description of the disa...,https://reliefweb.int/node/3354153,2019-10-18,reliefweb
2,india: monsoon floods emergency plan of action...,14 Sep 2019,a. situation analysis description of the disa...,https://reliefweb.int/node/3303423,2019-09-14,reliefweb
3,situation report on flood/heavy rainfall as on...,05 Sep 2019,1. forecast i. imd 05 september (day 1): ♦ he...,https://reliefweb.int/node/3292673,2019-09-05,reliefweb
4,situation report on flood/heavy rain fall as o...,02 Sep 2019,1. forecast i. imd • 02 september (day 1): ♦ ...,https://reliefweb.int/node/3287208,2019-09-02,reliefweb


### Create a column which combines the title and body 
A new column called "text" will be created which is the text from the "title" and "body" of each observation with a space separating them.

In [11]:
# Function to combine columns of text into one new text column for the dataframe
def combine_text(df, column1, column2):
    df["text"] = df[column1] + " " + df[column2]

In [12]:
# combine the title and body of each observation in the dataframe
combine_text(reliefweb06_df, "title", "body")

# check
reliefweb06_df.head()

,title,date,body,url,datetime,source,text
0,situation report on flood/heavy rain fall as o...,21 Oct 2019,ministry of home affairs disaster management ...,https://reliefweb.int/node/3365543,2019-10-21,reliefweb,situation report on flood/heavy rain fall as o...
1,india: monsoon rains and floods emergency plan...,18 Oct 2019,a. situation analysis description of the disa...,https://reliefweb.int/node/3354153,2019-10-18,reliefweb,india: monsoon rains and floods emergency plan...
2,india: monsoon floods emergency plan of action...,14 Sep 2019,a. situation analysis description of the disa...,https://reliefweb.int/node/3303423,2019-09-14,reliefweb,india: monsoon floods emergency plan of action...
3,situation report on flood/heavy rainfall as on...,05 Sep 2019,1. forecast i. imd 05 september (day 1): ♦ he...,https://reliefweb.int/node/3292673,2019-09-05,reliefweb,situation report on flood/heavy rainfall as on...
4,situation report on flood/heavy rain fall as o...,02 Sep 2019,1. forecast i. imd • 02 september (day 1): ♦ ...,https://reliefweb.int/node/3287208,2019-09-02,reliefweb,situation report on flood/heavy rain fall as o...


### Create "new" dataframe 
In preparation of eventually combining all the collected csv files into one, the format  will be a text (combined all text) column,a date (as a datetime type) column, and a source(naming where the data was pulled from) column. 
As a precaution against running into a copy warning, `.copy()` is added to the end of the operation. The copy function will create a copy of the columns of the dataframe and when either of dataframes is modified it will not affect the other dataframe. 

In [13]:
# making new dataframe from the previous one
# select the rows interested in: datetime (the date as datetime type), text, source
reliefweb_new = reliefweb06_df[['datetime', 'url', 'text']].copy()

In [14]:
# check
reliefweb_new.head()

,datetime,url,text
0,2019-10-21,https://reliefweb.int/node/3365543,situation report on flood/heavy rain fall as o...
1,2019-10-18,https://reliefweb.int/node/3354153,india: monsoon rains and floods emergency plan...
2,2019-09-14,https://reliefweb.int/node/3303423,india: monsoon floods emergency plan of action...
3,2019-09-05,https://reliefweb.int/node/3292673,situation report on flood/heavy rainfall as on...
4,2019-09-02,https://reliefweb.int/node/3287208,situation report on flood/heavy rain fall as o...


In [15]:
# check the types 
reliefweb_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 3 columns):
datetime    344 non-null datetime64[ns]
url         344 non-null object
text        344 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 8.2+ KB


### Rename column names 
To help with eventual combining of all the csv files (containing text data related to our database search), certain columns will be renamed to be uniform across all csv files. The columns will be a "date" column which will be a datetime type column of when the article was published, a "source" column which states the source of the data, and a "text" column which has the text data of the article (the title and article body).

In [16]:
# function to rename columns to "date", "source", "text"
def rename_col(df, column_date, column_source, column_text):
    df.rename(columns={column_date: "date",
                       column_source: "source", 
                       column_text :"text"}, 
              inplace = True)

In [17]:
# apply function to the dataframe
rename_col(reliefweb_new, "datetime", "url", "text")

In [19]:
# check
reliefweb_new.head()

,date,source,text
0,2019-10-21,https://reliefweb.int/node/3365543,situation report on flood/heavy rain fall as o...
1,2019-10-18,https://reliefweb.int/node/3354153,india: monsoon rains and floods emergency plan...
2,2019-09-14,https://reliefweb.int/node/3303423,india: monsoon floods emergency plan of action...
3,2019-09-05,https://reliefweb.int/node/3292673,situation report on flood/heavy rainfall as on...
4,2019-09-02,https://reliefweb.int/node/3287208,situation report on flood/heavy rain fall as o...


## Save the new dataframe to a csv
This "new" dataframe should only have the three expected columns and has all the cleaning done to the text. 

In [20]:
# create csv file from the new dataframe
reliefweb_new.to_csv("../data/reliefweb_clean.csv", index = False)

**Summary**: In this notebook a csv file was cleaned and processed to get it correctly formatted for text analysis. A new csv file was also created including the columns which will be required for the ultimate combined csv of all the csv files. In the next notebook, some intial text analysis will be done to try to find the most important sentences from the articles collected. 